referensi https://www.analyticsvidhya.com/blog/2021/04/forward-feature-selection-and-its-implementation/

In [1]:
# # instal library mlxtend
# %pip install mlxtend

In [2]:
# Import Library
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.feature_selection import mutual_info_regression,SelectKBest 
from sklearn.ensemble import RandomForestRegressor
from skfeature.function.similarity_based import fisher_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
#import dataset
df = pd.read_csv('/home/jupyter-17523142/Rama/Dataset/dataset_rama.csv')
X = df.drop(['No', 'Mango Cultivars', 'Vit C (mg/100g)', 'TA (mg/100g)', 'SSC (oBrix)', 'label'], axis = 1)
y_vitc = df.loc[:, 'Vit C (mg/100g)']

# separate dataset into train and test
x_train, x_test, y_train, y_test = train_test_split(
    X, y_vitc, test_size = 0.3, random_state=0)

In [4]:
%%time

#cari ranking setiap fitur berdasarkan fisher score
fisher_score = fisher_score.fisher_score(x_train.to_numpy(), y_train.to_numpy())

fisher_score

/home/jupyter-17523142/.local/lib/python3.7/site-packages/skfeature/function/similarity_based/fisher_score.py:59: RuntimeWarning: divide by zero encountered in true_divide
  score = 1.0 / lap_score - 1


CPU times: user 6.25 s, sys: 206 ms, total: 6.46 s
Wall time: 421 ms


array([ 778, 1022, 1031, ..., 1216, 1218,    0])

In [5]:
# Ascending
# bikin dataframe buat nyimpan fitur beserta rankingnya
fisher_ranking = pd.DataFrame(fisher_score)
features = pd.DataFrame(x_train.columns)
fisher_ranking = pd.concat([features,fisher_ranking],axis=1)

# Assign the column name
fisher_ranking.columns = ["Features","Fisher_Ranking"]

# Print features score
print(fisher_ranking.nsmallest(300,columns="Fisher_Ranking"))

     Features  Fisher_Ranking
1556   2500.2               0
260    1111.3               1
1188   1845.4               2
1406   2184.3               3
1189   1846.7               4
...       ...             ...
1493     2357             295
802    1447.6             296
801    1446.8             297
800      1446             298
798    1444.4             299

[300 rows x 2 columns]


In [6]:
x_train_selected = x_train[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:300, 0]]
x_test_selected = x_test[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:300, 0]]

In [7]:
# calling the Linear Regression model
lr_model =  LinearRegression()
sfs = sfs(lr_model, k_features=100, forward=False, verbose=2, scoring='neg_root_mean_squared_error', n_jobs=-1)

In [8]:
%%time

sfs = sfs.fit(x_train_selected, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 292 out of 300 | elapsed:    7.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    7.8s finished

[2022-07-14 22:18:48] Features: 299/100 -- score: -3509628.81994543[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 290 out of 299 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 299 out of 299 | elapsed:    1.6s finished

[2022-07-14 22:18:50] Features: 298/100 -- score: -2486034.5419186438[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 289 out of 298 | elapsed:    1.7s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 298 out of 298 | elapsed:    1.7

CPU times: user 22min 25s, sys: 25.4 s, total: 22min 50s
Wall time: 22min 34s


[Parallel(n_jobs=-1)]: Done 101 out of 101 | elapsed:    6.1s finished

[2022-07-14 22:41:14] Features: 100/100 -- score: -16354850935.989359

In [9]:
feat_names = list(sfs.k_feature_names_)
print("Fitur yang diambil adalah: ")
print(feat_names)
len(feat_names)

Fitur yang diambil adalah: 
['2500.2', '1845.4', '2184.3', '1857.3', '1838.8', '1820.7', '1837.5', '1905', '1896.7', '1913.5', '1874.7', '1885.6', '1891.1', '1723.9', '1726.2', '1732', '1789.3', '1798', '1805.5', '1768.6', '1785.6', '1923.4', '2102.8', '2109.6', '2067.6', '2116.5', '1936.3', '2000.6', '1496.1', '1489.2', '1478.2', '1538.7', '1425.4', '1419.9', '1455.8', '1465.6', '1442', '1669.5', '1671.7', '1706.9', '1711.4', '1612.4', '1623.5', '1493.5', '1600.5', '2350.6', '1595.5', '1592.6', '1591.6', '2418.6', '1586.7', '1621.5', '1620.5', '2414.1', '1615.4', '2411.8', '1611.4', '1584.8', '1565.7', '1561.9', '1560', '1559.1', '1557.2', '1551.6', '1563.8', '1566.6', '1580.9', '1578', '1574.2', '1573.3', '1570.4', '1569.5', '1567.6', '1625.5', '1684.7', '1679.2', '1678.1', '1674.9', '1709.1', '1672.7', '2385.2', '1645.1', '1643.1', '1637.9', '1632.7', '1648.3', '1668.4', '1666.3', '1665.2', '1664.1', '2376.5', '2374.3', '1652.5', '1651.4', '2365.6', '1585.8', '2361.3', '1448.5', '14

100

# Prediksi Linear Regression

In [10]:
#cross validation 10-fold
cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [11]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = [100]

for nfeat in n_feat:
        print("==================================================")
        
        #ambil n fitur input hasil seleksi fitur Fisher Score
        x_train_selected = x_train[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

        #Create a Linear Regression
        lr_model = LinearRegression()
        
        #Train the model using the training sets
        lr_model.fit(x_train_selected, y_train)
        y_pred_lr = lr_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(lr_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Linear Regression data Train dengan " + str(nfeat) + " fitur: " 
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Linear Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs((round(scores['train_r2'].mean(), 2)))))    
        print("----------------------------")
        print("MSE model Linear data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_lr), 2))))
        print("RMSE model Linear data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_lr, squared = False), 2))))
        print("MAE model Linear data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_lr), 2))))
        print("R2 model Linear data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_lr), 2))))
        print("==================================================")

MSE model Linear Regression data Train dengan 100 fitur: 23.35
RMSE model Linear Regression data Train dengan 100 fitur:4.82
MAE model Linear Regression data Train dengan 100 fitur:2.41
R2 model Linear Regression data Train dengan 100 fitur:0.88
----------------------------
MSE model Linear data Test dengan 100 fitur:9.66877660017087e+18
RMSE model Linear data Test dengan 100 fitur:3109465645.44
MAE model Linear data Test dengan 100 fitur:862348398.95
R2 model Linear data Test dengan 100 fitur:5.606161113028572e+16
CPU times: user 6.42 s, sys: 200 ms, total: 6.62 s
Wall time: 255 ms


# Prediksi Random Forest Regressor

## 100 Trees

In [12]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = [100]
n_trees = [100]

for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        
        #ambil n fitur input hasil seleksi fitur Fisher Score
        x_train_selected = x_train[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

        #Create a Random Forest Regression
        rfg_model = RandomForestRegressor(n_estimators=ntrees, random_state=100)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg = rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur: " 
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs((round(scores['train_r2'].mean(), 2)))))    
        print("----------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print("==================================================")

MSE model Linear Regression data Train dengan 100 fitur: 38.85
RMSE model Linear Regression data Train dengan 100 fitur:6.23
MAE model Linear Regression data Train dengan 100 fitur:4.75
R2 model Linear Regression data Train dengan 100 fitur:0.8
----------------------------
MSE model Linear data Test dengan 100 fitur:179.97
RMSE model Linear data Test dengan 100 fitur:13.42
MAE model Linear data Test dengan 100 fitur:10.52
R2 model Linear data Test dengan 100 fitur:0.04
CPU times: user 6.93 s, sys: 105 ms, total: 7.04 s
Wall time: 3.35 s


## 150 Trees

In [13]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = [100]
n_trees = [150]

for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        
        #ambil n fitur input hasil seleksi fitur Fisher Score
        x_train_selected = x_train[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nsmallest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

        #Create a Random Forest Regression
        rfg_model = RandomForestRegressor(n_estimators=ntrees, random_state=100)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg = rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur: " 
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs((round(scores['train_r2'].mean(), 2)))))    
        print("----------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print("==================================================")

MSE model Linear Regression data Train dengan 100 fitur: 38.49
RMSE model Linear Regression data Train dengan 100 fitur:6.2
MAE model Linear Regression data Train dengan 100 fitur:4.74
R2 model Linear Regression data Train dengan 100 fitur:0.8
----------------------------
MSE model Linear data Test dengan 100 fitur:180.07
RMSE model Linear data Test dengan 100 fitur:13.42
MAE model Linear data Test dengan 100 fitur:10.56
R2 model Linear data Test dengan 100 fitur:0.04
CPU times: user 4.78 s, sys: 15.3 ms, total: 4.8 s
Wall time: 4.79 s


## 200 Trees

In [14]:
%%time

#tentukan metode scoring yang digunakan
metrics = {'rmse': 'neg_root_mean_squared_error',
               'mse': 'neg_mean_squared_error',
               'mae': 'neg_mean_absolute_error',
               'r2': 'r2'} 

n_feat = [100]
n_trees = [200]
for nfeat in n_feat:
    for ntrees in n_trees:
        print("==================================================")
        
        #ambil n fitur input hasil seleksi fitur Fisher Score
        x_train_selected = x_train[fisher_ranking.nlargest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]
        x_test_selected = x_test[fisher_ranking.nlargest(300,columns="Fisher_Ranking").iloc[0:nfeat, 0]]

        #Create a Random Forest Regression
        rfg_model = RandomForestRegressor(n_estimators=ntrees, random_state=100)
        
        #Train the model using the training sets
        rfg_model.fit(x_train_selected, y_train)
        y_pred_rfg = rfg_model.predict(x_test_selected)

        #hitung score model dari data train
        scores = cross_validate(rfg_model, x_train_selected, y_train, scoring=metrics, cv=cv, return_train_score=True)

        print("MSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur: " 
              + str(abs(round(scores['train_mse'].mean(), 2))))
        print("RMSE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_rmse'].mean(), 2))))
        print("MAE model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs(round(scores['train_mae'].mean(), 2))))
        print("R2 model Random Forest Regression data Train dengan " + str(nfeat) + " fitur:"
              + str(abs((round(scores['train_r2'].mean(), 2)))))    
        print("----------------------------")
        print("MSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg), 2))))
        print("RMSE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_squared_error(y_test, y_pred_rfg, squared = False), 2))))
        print("MAE model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(mean_absolute_error(y_test, y_pred_rfg), 2))))
        print("R2 model Random Forest Regression data Test dengan " + str(nfeat) + " fitur:" 
          + str(abs(round(r2_score(y_test, y_pred_rfg), 2))))
        print("==================================================")

MSE model Linear Regression data Train dengan 100 fitur: 39.17
RMSE model Linear Regression data Train dengan 100 fitur:6.25
MAE model Linear Regression data Train dengan 100 fitur:4.77
R2 model Linear Regression data Train dengan 100 fitur:0.8
----------------------------
MSE model Linear data Test dengan 100 fitur:173.5
RMSE model Linear data Test dengan 100 fitur:13.17
MAE model Linear data Test dengan 100 fitur:10.42
R2 model Linear data Test dengan 100 fitur:0.01
CPU times: user 5.56 s, sys: 23.4 ms, total: 5.59 s
Wall time: 5.58 s
